## Predictive Modeling  Exploration: Logistic Regression with and without lasso and ridge regularization

In [ ]:
# !jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/###.ipynb" --output "/content/drive/MyDrive/Colab Notebooks/####.html"

In [ ]:
complete_data <- customer_year_summary %>%
  drop_na()

head(complete_data)

CUSTOMER_NUMBER,total_order_2023,total_order_2024,percent_change,growth_category,FIRST_DELIVERY_DATE,years_since_first_order,TRADE_CHANNEL.x,CO2_CUSTOMER,COLD_DRINK_CHANNEL,⋯,LOCAL_MARKET_PARTNER,FREQUENT_ORDER_TYPE,TRADE_CHANNEL.y,SALES REP,CALL CENTER,null,MYCOKE LEGACY,MYCOKE360,OTHER,EDI
<int>,<dbl>,<dbl>,<dbl>,<chr>,<date>,<dbl>,<fct>,<lgl>,<fct>,⋯,<lgl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
500245678,370.0,383.5,3.648649,grew,2018-03-19,6,OUTDOOR ACTIVITIES,TRUE,EVENT,⋯,TRUE,SALES REP,OUTDOOR ACTIVITIES,721,32.5,0,0.0,0.0,0,0
500245685,601.5,482.0,-19.866999,declined,2018-03-02,6,FAST CASUAL DINING,TRUE,DINING,⋯,TRUE,OTHER,FAST CASUAL DINING,0,212.5,0,871.0,0.0,0,0
500245689,546.5,621.0,13.632205,grew,2018-02-28,6,LICENSED HOSPITALITY,FALSE,DINING,⋯,TRUE,OTHER,LICENSED HOSPITALITY,0,0.0,0,929.0,238.5,0,0
500245690,325.0,300.5,-7.538462,declined,2018-03-01,6,FAST CASUAL DINING,TRUE,DINING,⋯,TRUE,SALES REP,FAST CASUAL DINING,1,10.0,0,516.5,98.0,0,0
500245695,1037.5,807.5,-22.168675,declined,2018-02-28,6,HEALTHCARE,TRUE,WELLNESS,⋯,TRUE,OTHER,HEALTHCARE,0,0.0,0,1562.5,282.5,0,0
500245698,282.0,1088.5,285.992908,grew,2018-03-02,6,OTHER DINING & BEVERAGE,FALSE,DINING,⋯,FALSE,OTHER,OTHER DINING & BEVERAGE,0,498.5,0,229.0,643.0,0,0


In [ ]:
install.packages('glmnet')
library(tidyverse)  # For data wrangling
library(caret)      # For machine learning utilities
library(glmnet)     # For regularized logistic regression (Lasso/Ridge)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘RcppEigen’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-8



In [ ]:
# Read in version of complete_data that is filtered to CO2 = False, Local Market Partner = True, and Fountain Only = True
complete_data <- read.csv('complete_data_subset.csv')

In [ ]:
# Change growth_category to a factor
complete_data <- complete_data %>%
  mutate(growth_category = factor(growth_category, levels = c("declined", "grew")))


In [ ]:
library(tidyverse)
library(caret)

# Ensure categorical variables are factors
complete_data <- complete_data %>%
  mutate(across(where(is.character), as.factor))

# Drop ID-like columns
model_data <- complete_data %>%
  select(-CUSTOMER_NUMBER, -TRADE_CHANNEL.y)

# Ensure target variable is a factor
model_data$growth_category <- as.factor(model_data$growth_category)

# Split data
set.seed(42)
train_index <- createDataPartition(model_data$growth_category, p = 0.8, list = FALSE)
train_data <- model_data[train_index, ]
test_data <- model_data[-train_index, ]

# Check for missing values and handle them
train_data <- na.omit(train_data)


In [ ]:
levels(as.factor(train_data$growth_category))


[1] "declined" "grew"

In [ ]:

train_data <- train_data %>%
  mutate(growth_category = as.factor(ifelse(growth_category == "grew", "yes", "no")))


In [ ]:
colSums(is.na(train_data))


total_order_2023    total_order_2024      percent_change     growth_category 
                  0                   0                   0                   0 
    TRADE_CHANNEL.x  COLD_DRINK_CHANNEL   SUB_TRADE_CHANNEL FREQUENT_ORDER_TYPE 
                  0                   0                   0                   0 
          SALES.REP         CALL.CENTER                null       MYCOKE.LEGACY 
                  0                   0                   0                   0 
          MYCOKE360               OTHER                 EDI         cases_total 
                  0                   0                   0                   0 
     fountain_total 
                  0

In [ ]:
sapply(train_data, function(x) sum(is.infinite(x) | is.nan(x)))


total_order_2023    total_order_2024      percent_change     growth_category 
                  0                   0                   0                   0 
    TRADE_CHANNEL.x  COLD_DRINK_CHANNEL   SUB_TRADE_CHANNEL FREQUENT_ORDER_TYPE 
                  0                   0                   0                   0 
          SALES.REP         CALL.CENTER                null       MYCOKE.LEGACY 
                  0                   0                   0                   0 
          MYCOKE360               OTHER                 EDI         cases_total 
                  0                   0                   0                   0 
     fountain_total 
                  0

In [ ]:
train_data$percent_change[is.infinite(train_data$percent_change)] <- NA
train_data$percent_change[is.nan(train_data$percent_change)] <- NA


In [ ]:
train_data$percent_change[is.na(train_data$percent_change)] <- median(train_data$percent_change, na.rm = TRUE)


In [ ]:
sapply(train_data, function(x) sum(is.infinite(x) | is.nan(x)))

total_order_2023    total_order_2024      percent_change     growth_category 
                  0                   0                   0                   0 
    TRADE_CHANNEL.x  COLD_DRINK_CHANNEL   SUB_TRADE_CHANNEL FREQUENT_ORDER_TYPE 
                  0                   0                   0                   0 
          SALES.REP         CALL.CENTER                null       MYCOKE.LEGACY 
                  0                   0                   0                   0 
          MYCOKE360               OTHER                 EDI         cases_total 
                  0                   0                   0                   0 
     fountain_total 
                  0

In [ ]:
# Original model
logistic_model <- glm(growth_category ~ ., data = train_data, family = binomial)
summary(logistic_model)


Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”



Call:
glm(formula = growth_category ~ ., family = binomial, data = train_data)

Coefficients: (30 not defined because of singularities)
                                               Estimate Std. Error z value
(Intercept)                                   6.836e+02  6.769e+05   0.001
total_order_2023                             -3.454e+00  9.034e+02  -0.004
total_order_2024                              3.385e+00  1.158e+03   0.003
percent_change                                2.203e+00  1.178e+03   0.002
TRADE_CHANNEL.xACTIVITIES                    -1.104e+02  4.914e+05   0.000
TRADE_CHANNEL.xBULK TRADE                    -1.023e+03  5.290e+05  -0.002
TRADE_CHANNEL.xCOMPREHENSIVE DINING          -4.431e+00  2.155e+05   0.000
TRADE_CHANNEL.xEDUCATION                     -2.892e+01  4.175e+05   0.000
TRADE_CHANNEL.xFAST CASUAL DINING             9.276e+00  4.027e+05   0.000
TRADE_CHANNEL.xGENERAL RETAILER               1.952e+02  4.745e+07   0.000
TRADE_CHANNEL.xGOURMET FOOD RETAILER  

In [ ]:
table(train_data$growth_category)


 no yes 
265 192 

In [ ]:
library(glmnet)

# Prepare the data
X <- model.matrix(growth_category ~ ., data = train_data)[, -1]  # Remove intercept
y <- train_data$growth_category  # Response variable

# Convert y to a factor (if needed)
y <- as.factor(y)

# Define a sequence of lambda values to test
lambda_seq <- 10^seq(3, -3, by = -0.1)

# Fit Lasso Logistic Regression (alpha = 1 for Lasso)
lasso_model <- cv.glmnet(X, y, family = "binomial", alpha = 1, lambda = lambda_seq)

# Fit Ridge Logistic Regression (alpha = 0 for Ridge)
ridge_model <- cv.glmnet(X, y, family = "binomial", alpha = 0, lambda = lambda_seq)

# Find optimal lambda values
best_lambda_lasso <- lasso_model$lambda.min
best_lambda_ridge <- ridge_model$lambda.min

# Print best lambda values
print(paste("Best Lambda for Lasso:", best_lambda_lasso))
print(paste("Best Lambda for Ridge:", best_lambda_ridge))

# Get coefficients for the best Lasso model
lasso_coef <- coef(lasso_model, s = "lambda.min")
print(lasso_coef)

# Get coefficients for the best Ridge model
ridge_coef <- coef(ridge_model, s = "lambda.min")
print(ridge_coef)

# Prepare the test data and ensure no NA values in the response variable
# Subset test_data to remove rows with NA in 'growth_category' (if any)
test_data_subset <- test_data[!is.na(test_data$growth_category), ]

# Check that both test data and predictions align
X_test <- model.matrix(growth_category ~ ., data = test_data_subset)[, -1]
y_test <- test_data_subset$growth_category

# Predict on test data
lasso_preds <- predict(lasso_model, s = best_lambda_lasso, newx = X_test, type = "response")
ridge_preds <- predict(ridge_model, s = best_lambda_ridge, newx = X_test, type = "response")

# Convert predictions to binary outcome (ensure same length)
lasso_class <- ifelse(lasso_preds > 0.5, 1, 0)
ridge_class <- ifelse(ridge_preds > 0.5, 1, 0)

# Ensure that the predicted and actual labels have the same length
lasso_class <- lasso_class[1:length(y_test)]
ridge_class <- ridge_class[1:length(y_test)]

# Evaluate accuracy (assuming test_data$growth_category is the true label)
lasso_accuracy <- mean(lasso_class == y_test)
ridge_accuracy <- mean(ridge_class == y_test)

# Print the accuracies
print(paste("Lasso Accuracy:", lasso_accuracy))
print(paste("Ridge Accuracy:", ridge_accuracy))


[1] "Best Lambda for Lasso: 0.001"
[1] "Best Lambda for Ridge: 0.001"
70 x 1 sparse Matrix of class "dgCMatrix"
                                                        s1
(Intercept)                                  -4.643484e-01
total_order_2023                             -2.616011e-04
total_order_2024                              .           
percent_change                                3.705999e-01
TRADE_CHANNEL.xACTIVITIES                     .           
TRADE_CHANNEL.xBULK TRADE                     .           
TRADE_CHANNEL.xCOMPREHENSIVE DINING           3.832439e-01
TRADE_CHANNEL.xEDUCATION                      3.335096e+00
TRADE_CHANNEL.xFAST CASUAL DINING             .           
TRADE_CHANNEL.xGENERAL RETAILER               .           
TRADE_CHANNEL.xGOURMET FOOD RETAILER          .           
TRADE_CHANNEL.xHEALTHCARE                     2.592711e+00
TRADE_CHANNEL.xINDUSTRIAL                     3.910892e-01
TRADE_CHANNEL.xLICENSED HOSPITALITY           .           
TRA

In [ ]:
# Second lasso/ridge logistic
library(glmnet)

# Prepare the data
X <- model.matrix(growth_category ~ ., data = train_data)[, -1]  # Remove intercept
y <- train_data$growth_category  # Response variable

# Convert y to a factor (if needed)
y <- as.factor(y)

# Define a sequence of lambda values to test
lambda_seq <- 10^seq(3, -3, by = -0.1)

# Fit Lasso Logistic Regression (alpha = 1 for Lasso)
lasso_model <- cv.glmnet(X, y, family = "binomial", alpha = 1, lambda = lambda_seq)

# Fit Ridge Logistic Regression (alpha = 0 for Ridge)
ridge_model <- cv.glmnet(X, y, family = "binomial", alpha = 0, lambda = lambda_seq)

# Find optimal lambda values
best_lambda_lasso <- lasso_model$lambda.1se
best_lambda_ridge <- ridge_model$lambda.1se

# Print best lambda values
print(paste("Best Lambda for Lasso:", best_lambda_lasso))
print(paste("Best Lambda for Ridge:", best_lambda_ridge))

# Get coefficients for the best Lasso model
lasso_coef <- coef(lasso_model, s = "lambda.min")
print(lasso_coef)

# Get coefficients for the best Ridge model
ridge_coef <- coef(ridge_model, s = "lambda.min")
print(ridge_coef)

# Prepare the test data and ensure no NA values in the response variable
# Subset test_data to remove rows with NA in 'growth_category' (if any)
test_data_subset <- test_data[!is.na(test_data$growth_category), ]

# Check that both test data and predictions align
X_test <- model.matrix(growth_category ~ ., data = test_data_subset)[, -1]
y_test <- test_data_subset$growth_category

# Predict on test data
lasso_preds <- predict(lasso_model, s = best_lambda_lasso, newx = X_test, type = "class")
ridge_preds <- predict(ridge_model, s = best_lambda_ridge, newx = X_test, type = "class")

# Convert predictions to binary outcome (ensure same length)
lasso_class <- ifelse(lasso_preds > 0.5, 1, 0)
ridge_class <- ifelse(ridge_preds > 0.5, 1, 0)

# Ensure that the predicted and actual labels have the same length
lasso_class <- lasso_class[1:length(y_test)]
ridge_class <- ridge_class[1:length(y_test)]

# Evaluate accuracy (assuming test_data$growth_category is the true label)
lasso_accuracy <- mean(lasso_class == y_test)
ridge_accuracy <- mean(ridge_class == y_test)

# Print the accuracies
print(paste("Lasso Accuracy:", lasso_accuracy))
print(paste("Ridge Accuracy:", ridge_accuracy))


[1] "Best Lambda for Lasso: 0.00199526231496888"
[1] "Best Lambda for Ridge: 0.00125892541179417"
70 x 1 sparse Matrix of class "dgCMatrix"
                                                        s1
(Intercept)                                  -4.643484e-01
total_order_2023                             -2.616011e-04
total_order_2024                              .           
percent_change                                3.705999e-01
TRADE_CHANNEL.xACTIVITIES                     .           
TRADE_CHANNEL.xBULK TRADE                     .           
TRADE_CHANNEL.xCOMPREHENSIVE DINING           3.832439e-01
TRADE_CHANNEL.xEDUCATION                      3.335096e+00
TRADE_CHANNEL.xFAST CASUAL DINING             .           
TRADE_CHANNEL.xGENERAL RETAILER               .           
TRADE_CHANNEL.xGOURMET FOOD RETAILER          .           
TRADE_CHANNEL.xHEALTHCARE                     2.592711e+00
TRADE_CHANNEL.xINDUSTRIAL                     3.910892e-01
TRADE_CHANNEL.xLICENSED HOSPITALIT